<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%A0%95%EC%88%98%EC%9D%B8_%EC%84%9C%EB%B9%84%EC%8A%A42%EC%9E%91%EC%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/"
SEED = 42

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00


In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=e43da083612354cf69937117182727476cb956856147a11a9998a54556718bf4
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


데이터 정제 및 전처리

In [ ]:
rawyogiyo_df = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")
yogiyo_df = rawyogiyo_df.copy()
yogiyo_df

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...
...,...,...,...,...,...,...
18681,맛있게 잘 먹었어요^^,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요… 그래도 잘 먹었습니다,4,5,5,5.0,ㅠㅠ 죄송합니다 !! 다음에도 애용해주세용 ㅎㅎ
18683,배달 빨라요 맛있게 잘 먹었습니당 !,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ
18684,잘 먹었어요~~~~~,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ


In [ ]:
yogiyo_df = yogiyo_df[["고객리뷰", "사장댓글"]]

In [ ]:
yogiyo_df["고객리뷰"] = yogiyo_df["고객리뷰"].str.replace("[^A-Za-z가-힣0-9 .,!?]" , " ",regex=True)
yogiyo_df["사장댓글"] = yogiyo_df["사장댓글"].str.replace("[^A-Za-z가-힣0-9 .,!?]" , " ",regex=True)

yogiyo_df

<ipython-input-10-ee080adb76aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["고객리뷰"] = yogiyo_df["고객리뷰"].str.replace("[^A-Za-z가-힣0-9 .,!?]" , " ",regex=True)
<ipython-input-10-ee080adb76aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["사장댓글"] = yogiyo_df["사장댓글"].str.replace("[^A-Za-z가-힣0-9 .,!?]" , " ",regex=True)


,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,감사합니다!!! 다음에도 이용해주세용


In [ ]:
yogiyo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18686 entries, 0 to 18685
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고객리뷰    18686 non-null  object
 1   사장댓글    18686 non-null  object
dtypes: object(2)
memory usage: 292.1+ KB


In [ ]:
yogiyo_df

,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,감사합니다!!! 다음에도 이용해주세용


가게 이름 제거 , ** 님 고객님으로 수정

In [ ]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]
len(ygy_store_lst)

125

In [ ]:
ygy_store_lst = [re.split("[-,& ]", i ) for i in ygy_store_lst]

In [ ]:
ygy_store_lst = sum(ygy_store_lst, [])

In [ ]:
ygy_store_lst

['1988응답하라추억의옛날도시락',
 '방이점',
 '워커스커피',
 '로스터스',
 '떡깨비',
 '가락점',
 '당치땡',
 '헬리오시티점',
 '우시장국밥',
 '석촌점',
 '고돼지',
 '송파점',
 '마라의신마라탕',
 '마라샹궈',
 '춘리마라탕',
 '송파점',
 '손수경의육회담은연어',
 '가락점',
 '무국적식탁',
 '송파점',
 '청년치킨',
 '삼전점',
 '송파만다린',
 '배달의쌀국수',
 '송파점',
 '닭강점',
 '헬리오시티점',
 '토핑폭탄김치찜',
 '김치찌개',
 '송파점',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '송파점',
 '더티베지',
 '잭아저씨족발보쌈',
 '본점',
 '육회',
 '연어',
 '미아본점',
 '갈비민족',
 '본점',
 '구구족',
 '성신여대역점',
 '피자보이시나',
 '대학로점',
 '왕빈자삼파전',
 '미아본점',
 '행복한찜닭',
 '성북점',
 '스시사소우',
 '저팔계',
 '중국',
 '마라탕',
 '덮밥',
 '요리',
 '전문점',
 '명작파스타',
 '성북점',
 '올데이파스타',
 '동대문구점',
 '대한냉면',
 '성북점',
 '인생제육',
 '고기듬뿍미트박스',
 '성북점',
 '피자스웨그',
 '성북점',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '북서울꿈의숲점',
 '더팔당매운오징어',
 '닭발',
 '김치찜',
 '우리집반찬도시락',
 '혜화막내딸지점',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '마약볶음밥',
 '대찌',
 '대파생고기김치찌개',
 '정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화

In [ ]:

del_reply = []
for reply in yogiyo_df["사장댓글"]:
    reply_words = reply.split()
    new_reply_words = []
    for reply_word in reply_words:
        if not any(store in reply_word for store in ygy_store_lst): new_reply_words.append(reply_word)

    new_reply = ' '.join(new_reply_words)
    del_reply.append(new_reply)

del_reply

['이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다 항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다',
 '좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로가 싹 사라지네요! 주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요 오늘도 화이팅 있게 마무리하시길 바라요',
 '소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다..!! 저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요??전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요 더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?? 저희 메뉴 칭찬해 주셔서 감사합니다 고객님!! 늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요 항상 노력하고 발전하도록 하겠습니다 항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다',
 '저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다 고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다! 언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!! 매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다 하시는 일 모두 번창하시길 진심으로 바라고 있을게요',


In [ ]:
yogiyo_df["사장댓글"] = del_reply
yogiyo_df

<ipython-input-18-a596bb12a52c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["사장댓글"] = del_reply


,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신 ...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,감사합니다!!! 다음에도 이용해주세용


In [ ]:
yogiyo_df["사장댓글"] = yogiyo_df["사장댓글"].str.replace(r'\w+\s*님', '고객님', regex=True)
yogiyo_df

<ipython-input-19-a28c6ff86aaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["사장댓글"] = yogiyo_df["사장댓글"].str.replace(r'\w+\s*님', '고객님', regex=True)


,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신 ...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,감사합니다!!! 다음에도 이용해주세용


In [ ]:
yogiyo_df["고객리뷰"] = [' '.join(i.split()) for i in yogiyo_df["고객리뷰"]]
yogiyo_df["사장댓글"] = [' '.join(i.split()) for i in yogiyo_df["사장댓글"]]

<ipython-input-20-5e998ffcb67d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["고객리뷰"] = [' '.join(i.split()) for i in yogiyo_df["고객리뷰"]]
<ipython-input-20-5e998ffcb67d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo_df["사장댓글"] = [' '.join(i.split()) for i in yogiyo_df["사장댓글"]]


In [ ]:
yogiyo_df

,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신 ...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,감사합니다!!! 다음에도 이용해주세용


In [ ]:
# from kiwipiepy import Kiwi
# kiwi = Kiwi()

In [ ]:
# review = kiwi.tokenize(df["content"])
# reply = kiwi.tokenize(df["replyContent"])

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "skt/kogpt2-base-v2"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=2048)

In [ ]:
class ReplyDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.question = df["고객리뷰"].tolist()
        self.answer = df["사장댓글"].tolist()

    def __len__(self):
        return len(self.question)

    def __getitem__(self,idx):
        return "<q>" + self.question[idx] + "</s><a>" + self.answer[idx] + "</s>"

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt",padding=True)
    return {"x":x}

In [ ]:
dt = ReplyDataset(yogiyo_df)
dl = torch.utils.data.DataLoader(dt, batch_size=2, collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 27013,  9518,  9137,  9984,  9358,  7609, 11355,
          49067,  8084, 22311, 15729, 12268,  8084, 28005, 10785,  7235, 16317,
          22353,  7788, 11219,  8236,  7816,  8006,  8084,     1,  9724,   439,
          28886, 10431,  9265,  8806, 13669, 10345,  8236,  7810,  7788, 15940,
           7281,  7483,  7172,  7182,   376,   376, 11355, 11178,  9554,  9164,
          10897, 10997,  7235,  9432, 41014, 40492, 21291, 11611, 30782,  9019,
          16913,  7182,  9548, 18401,  9136,  9723,  7071,  7788, 12298, 19495,
          13489, 35359,  9207,  9716, 10805, 26616, 16913,  7182, 12086, 11342,
          42226,  6958,  7888,  9075, 10070,  7235, 29080, 12557, 11219,  7888,
           6962, 13158,  6872, 16913,  7182,     1,     3,     3],
         [ 9724,   455,   405,  7478,  7685,  6824, 13289,  8017,  7098,  8084,
           9705, 11355, 18479,  9443,  9784,  8017, 16691,  9108,  9039,  8514,
           6866,  8084,   376,   3

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:,:-1]
        pred = pred.reshape(-1,n_class)

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten()

        mask = tgt != 3
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
batch_size = 2
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

In [ ]:
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ReplyDataset(yogiyo_df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers = 8 )

for _ in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(train_loss)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/9343 [00:00<?, ?it/s]

1.647545783687114


  0%|          | 0/9343 [00:00<?, ?it/s]

1.2795127534436028


...

0.6401425850116427


  0%|          | 0/9343 [00:00<?, ?it/s]

0.5727710753597935


  0%|          | 0/9343 [00:00<?, ?it/s]

0.511990322652388


In [ ]:
model.save_pretrained(f"{DATA_PATH}kogpt2_chat")